In [1]:

import torch
import torchvision
from torch import Tensor
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

import os
import zipfile
from torch.utils.data import Dataset
import json 
from PIL import Image
import gc
import math
from collections import OrderedDict

import sys
import ctypes
import _ctypes#the hell is this?
import subprocess
import shutil
import time

cpu_device = torch.device('cpu')
cuda_device = torch.device('cuda')
DTYPENET = torch.float32
DTYPEDATA = torch.float32 #maybe even float16. Sadly it doesn't do anything
DEVICE = cpu_device
BATCH_SIZE = 256
LR = 1e-4

#if torch.cuda.is_available():
    #DEVICE = cuda_device
    #torch.cuda.empty_cache()
print(DEVICE)

padding00 = 1
ALLLAYERS = True


MAXARRLEN = 64*(32+2*padding00)*(32+2*padding00)
STARRLEN = 3*(32+2*padding00)*(32+2*padding00)
INPUTFILENAME = "inputTensorPaddedFlat.txt"
PADMASKFILENAME = "paddingmask.txt"
OUTPUTFILENAME = "output.txt"
LOGFILENAME = "log.txt"
ARRFROMWHICHTOREADOUTPUT = "B"

cpu


In [2]:
class BitConv2d(nn.Conv2d):
    def __init__(self, *args, num_bits: int = 8, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_bits = num_bits

        self.eps:float = 1e-5
        self.quantization_range: int = 2 ** (num_bits - 1) # Q_b in the paper


    def ste_weights(self, weights_gamma: float) -> Tensor:
        eps: float = 1e-7
        scaled_weights:Tensor = self.weight / (weights_gamma + eps)
        bin_weights_no_grad: Tensor = torch.clamp(torch.round(scaled_weights), min=-1, max=1)
        bin_weights_with_grad: Tensor = (bin_weights_no_grad - self.weight).detach() + self.weight
        return bin_weights_with_grad


    def binarize_weights(self, weights_gamma: float) -> Tensor:
        binarized_weights = self.ste_weights(weights_gamma)
        return binarized_weights


    def quantize_activations(self, _input:Tensor, input_gamma: float) -> Tensor:
        # Equation 4 BitNet paper
        quantized_input = torch.clamp(
                _input * self.quantization_range / input_gamma,
                -self.quantization_range + self.eps,
                self.quantization_range - self.eps,
            )
        return quantized_input


    def dequantize_activations(self, _input: Tensor, input_gamma: float, beta: float) -> Tensor:
        return _input * input_gamma * beta / self.quantization_range


    def forward(self, _input: Tensor) -> Tensor:
        normalized_input: Tensor = nn.functional.layer_norm(_input, (_input.shape[1:]))
        input_gamma: float = normalized_input.abs().max().item()
        weight_abs_mean: float = self.weight.abs().mean().item()

        binarized_weights = self.binarize_weights(weight_abs_mean)
        input_quant = self.quantize_activations(normalized_input, input_gamma)
        output = torch.nn.functional.conv2d(
            input=input_quant,
            weight=binarized_weights,
            bias=self.bias,
            stride=self.stride,
            padding=self.padding,
            dilation=self.dilation,
            groups=self.groups
        )#input=input_quant
        output = self.dequantize_activations(output, input_gamma, weight_abs_mean)

        return output
    
class BitLinear(nn.Linear):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        bias: bool = True,
        num_bits: int = 8,
    ):
        super().__init__(in_features, out_features, bias)
        self.eps:float = 1e-5
        self.quantization_range: int = 2 ** (num_bits - 1) # Q_b in the paper


    def ste_weights(self, weights_gamma: float) -> Tensor:
        eps: float = 1e-7
        scaled_weights:Tensor = self.weight / (weights_gamma + eps)
        bin_weights_no_grad: Tensor = torch.clamp(torch.round(scaled_weights), min=-1, max=1)
        bin_weights_with_grad: Tensor = (bin_weights_no_grad - self.weight).detach() + self.weight
        return bin_weights_with_grad


    def binarize_weights(self, weights_gamma: float) -> Tensor:
        binarized_weights = self.ste_weights(weights_gamma)
        return binarized_weights


    def quantize_activations(self, _input:Tensor, input_gamma: float) -> Tensor:
        # Equation 4 BitNet paper
        quantized_input = torch.clamp(
                _input * self.quantization_range / input_gamma,
                -self.quantization_range + self.eps,
                self.quantization_range - self.eps,
            )
        return quantized_input


    def dequantize_activations(self, _input: Tensor, input_gamma: float, beta: float) -> Tensor:
        return _input * input_gamma * beta / self.quantization_range


    def forward(self, _input: Tensor) -> Tensor:
        normalized_input: Tensor =nn.functional.layer_norm(_input,(_input.shape[1:]))
        input_gamma: float = normalized_input.abs().max().item()
        weight_abs_mean: float = self.weight.abs().mean().item()

        binarized_weights = self.binarize_weights(weight_abs_mean)
        input_quant = self.quantize_activations(normalized_input, input_gamma)
        output = torch.nn.functional.linear(input_quant, binarized_weights, self.bias)#input_quant
        output = self.dequantize_activations(output, input_gamma, weight_abs_mean)

        return output

class CNNBLOCK_DS(nn.Module):#i should use more *ags and **kwargs
    def __init__(self, in_channels_, out_channels_,
                 kernel_size_=3, stride_=1, 
                 padding_=1, bias_=False):
        super().__init__()
        self.conv_depth = BitConv2d(in_channels = in_channels_, 
                                    out_channels =in_channels_, 
                                    kernel_size =kernel_size_, 
                                    stride = stride_, 
                                    padding = padding_, 
                                    dilation = 1,
                                    groups =in_channels_,
                                    bias = bias_)
        self.conv_sep = BitConv2d(in_channels = in_channels_, 
                                  out_channels = out_channels_, 
                                  kernel_size=1, 
                                  stride = 1,
                                  padding= 0, 
                                  dilation=1,
                                  groups = 1,
                                  bias=False)
        self.lrlu = nn.LeakyReLU(0.1)
    def forward(self, x):
        return self.lrlu(self.conv_sep(self.conv_depth(x)))#i think we need bn here
        

class RESBLOCK(nn.Module):
    def __init__(self, list_of_params):
        super().__init__()
        modules = []
        for block in list_of_params:
            in_channels_ = block[0]
            out_channels_ = block[1]
            modules.append(CNNBLOCK_DS(in_channels_, out_channels_))
        self.suka = nn.Sequential(*modules)
    def forward(self, x):
        return x + self.suka(x)

class YOLONET(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2,2)
        
        self.convBlock1 = CNNBLOCK_DS(3, 64)
        
        #we need 5 32 layers, 5 64 layers, 5 128 layers and 5 256 layers. Don't forget to maxpool
        self.resblock64_1 = RESBLOCK([[64, 64], [64,64] ])#how many layers should i have in my resblock???
        self.resblock64_2 = RESBLOCK([[64, 64], [64,64] ])
        self.resblock64_3 = RESBLOCK([[64, 64], [64,64] ])
        self.resblock64_4 = RESBLOCK([[64, 64], [64,64] ])
        self.resblock64_5 = RESBLOCK([[64, 64], [64,64] ])

        #how do we upscale??
        self.convInter_64_128 = CNNBLOCK_DS(64, 128)
        self.resblock128_1 = RESBLOCK([[128, 128], [128, 128]])
        self.resblock128_2 = RESBLOCK([[128, 128], [128, 128]])
        self.resblock128_3 = RESBLOCK([[128, 128], [128, 128]])
        self.resblock128_4 = RESBLOCK([[128, 128], [128, 128]])
        self.resblock128_5 = RESBLOCK([[128, 128], [128, 128]])
        self.convInter_128_256 = CNNBLOCK_DS(128, 256)
        self.resblock256_1 = RESBLOCK([[256, 256], [256, 256]])
        self.resblock256_2 = RESBLOCK([[256, 256], [256, 256]])
        self.resblock256_3 = RESBLOCK([[256, 256], [256, 256]])
        self.resblock256_4 = RESBLOCK([[256, 256], [256, 256]])
        self.resblock256_5 = RESBLOCK([[256, 256], [256, 256]])
        self.resblock256_6 = RESBLOCK([[256, 256], [256, 256]])
        
        #self.convLast = nn.Conv2d(128, 16, 1 padding=0, bias=False)#how do we ensure we have [:, 16, 13, 13] output tensor????
        self.avgPool = nn.AvgPool2d(2,3)
        self.fc = BitLinear(256, 10)

    def forward(self,x):
        x = self.convBlock1(x)
        x = self.resblock64_1(x)
        x = self.resblock64_2(x)
        x = self.resblock64_3(x)
        x = self.pool(x)
        x = self.resblock64_4(x)
        x = self.resblock64_5(x)
        x = self.pool(x)
        x = self.convInter_64_128(x)
        x = self.resblock128_1(x)
        x = self.resblock128_2(x)
        x = self.pool(x)
        x = self.resblock128_3(x)
        x = self.pool(x)
        x = self.resblock128_4(x)
        x = self.resblock128_5(x)
        x = self.pool(x)
        x = self.convInter_128_256(x)
        x = self.resblock256_1(x)
        x = self.resblock256_2(x)
        x = self.resblock256_3(x)
        x = self.resblock256_4(x)
        x = self.resblock256_5(x)
        x = self.resblock256_6(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        return x

In [3]:
getLayerDimNet = YOLONET()
toSeeLayerDim = torch.squeeze(getLayerDimNet(torch.rand((1,3,32,32))) , 0)
print("SUKAAAA", toSeeLayerDim.shape)
padded00 = toSeeLayerDim.clone()
if ALLLAYERS == False:
    padded00 = torch.zeros((toSeeLayerDim.shape[0], toSeeLayerDim.shape[1]+2*padding00, toSeeLayerDim.shape[2]+2*padding00))
    padded00[:,padding00:padding00+toSeeLayerDim.shape[1],padding00:padding00+toSeeLayerDim.shape[2]] = toSeeLayerDim
OUTPUTARRAYLEN = len(padded00.flatten())
print(OUTPUTARRAYLEN)

SUKAAAA torch.Size([10])
10


In [4]:
class BitConv2dBitProba(nn.Conv2d):
    def __init__(self,weightAbsMean: float, *args,num_bits: int = 8, **kwargs):
        super().__init__(*args, **kwargs)
        self.weightAbsMean = weightAbsMean
        self.num_bits = num_bits
        self.eps:float = 1e-5
        self.quantization_range: int = 2 ** (num_bits - 1) # Q_b in the paper
    def quantize_activations(self, _input:Tensor, input_gamma: float) -> Tensor:
        # Equation 4 BitNet paper
        quantized_input = torch.clamp(
                _input * self.quantization_range / input_gamma,
                -self.quantization_range + self.eps,
                self.quantization_range - self.eps,)
        return quantized_input
    def dequantize_activations(self, _input: Tensor, input_gamma: float, beta: float) -> Tensor:
        return _input * input_gamma * beta / self.quantization_range
    def forward(self, _input: Tensor) -> Tensor:
        normalized_input: Tensor = nn.functional.layer_norm(_input, (_input.shape))
        input_gamma: float = normalized_input.abs().max().item()
        weight_abs_mean: float = self.weight.abs().mean().item()#i guess here is the crux of the problem
        input_quant = self.quantize_activations(normalized_input, input_gamma)
        output = torch.nn.functional.conv2d(
            input=input_quant,
            weight=self.weight,
            bias=self.bias,
            stride=self.stride,
            padding=self.padding,
            dilation=self.dilation,
            groups=self.groups)
        output = self.dequantize_activations(output, input_gamma, self.weightAbsMean)
        return output

class BitLinearBitProba(nn.Linear):
    def __init__(
        self,
        weightAbsMean: float,
        in_features: int,
        out_features: int,
        bias: bool = True,
        num_bits: int = 8,
    ):#i've no idea if the original network has bias in here
        super().__init__(in_features, out_features, bias)
        self.eps:float = 1e-5
        self.weightAbsMean = weightAbsMean
        self.quantization_range: int = 2 ** (num_bits - 1) # Q_b in the paper
    def quantize_activations(self, _input:Tensor, input_gamma: float) -> Tensor:
        # Equation 4 BitNet paper
        quantized_input = torch.clamp(
                _input * self.quantization_range / input_gamma,
                -self.quantization_range + self.eps,
                self.quantization_range - self.eps,)
        return quantized_input
    def dequantize_activations(self, _input: Tensor, input_gamma: float, beta: float) -> Tensor:
        return _input * input_gamma * beta / self.quantization_range
    def forward(self, _input: Tensor) -> Tensor:
        normalized_input: Tensor = nn.functional.layer_norm(_input, (_input.shape))
        input_gamma: float = normalized_input.abs().max().item()
        weight_abs_mean: float = self.weight.abs().mean().item()
        input_quant = self.quantize_activations(normalized_input, input_gamma)
        output = torch.nn.functional.linear(input_quant, self.weight, self.bias)
        output = self.dequantize_activations(output, input_gamma, self.weightAbsMean)   
        return output




In [5]:
zah0 = f'''
#include <stdio.h>
#include <stdbool.h>
#include <math.h>
#include <time.h>
#include <immintrin.h>
#include <pthread.h>
#include <string.h>
#define MAXARRLEN {MAXARRLEN}
#define STARRLEN {STARRLEN}
#define OUTPUTARRAYLEN {OUTPUTARRAYLEN}
#define INPUTFILENAME "{INPUTFILENAME}"
#define PADMASKFILENAME "{PADMASKFILENAME}"
#define OUTPUTFILENAME "{OUTPUTFILENAME}"
#define LOGFILENAME "{LOGFILENAME}"
bool P[MAXARRLEN];
float A[MAXARRLEN],B[MAXARRLEN],C[MAXARRLEN];
float absmax;
long int i;
clock_t st, end;
double time_spent_in_conv = 0.0;

/*
INSERT YOUR BIAS DECLARATION CODE HERE
*/



float min(float a,float b){{
  if (a <= b){{return a;}}
  else{{return b;}}
}}
float max(float a, float b){{
  if (a >= b){{return a;}}
  else{{return b;}}
}}
long int iib(int d,int i,int j,int H,int W){{return d*(H*W)+i*W+j;}}


void padNext(float NEXTARRAY[],bool PADDINGMASK[],int padding,int D,int H,int W){{
    long int car = 0;
    for (int d = 0; d < D; d++){{
        memset(&NEXTARRAY[car], 0, W*padding*sizeof(float));
        memset(&PADDINGMASK[car], 0, W*padding*sizeof(bool));
        car += W*padding;
        for (int h = padding; h < H - padding; h++){{
            memset(&NEXTARRAY[car], 0, padding*sizeof(float));
            memset(&PADDINGMASK[car], 0, padding*sizeof(bool));
            car += padding;
            car += W - 2*padding;
            memset(&NEXTARRAY[car], 0, padding*sizeof(float));
            memset(&PADDINGMASK[car], 0, padding*sizeof(bool));
            car += padding;
        }}
        memset(&NEXTARRAY[car], 0, W*padding*sizeof(float));
        memset(&PADDINGMASK[car], 0, W*padding*sizeof(bool));
        car += W*padding;
    }}
}}

float layernormQuantizationBiasNextpadding(int inputD, int inputH, int inputW,
                                           int out_channels,int stride,
                                           float INPUTARRAY[],bool PADDINGMASK[],
                                           float BIAS[],float OUTPUTARRAY[],
                                           int output_padding,bool bias_flag,
                                           int kernelSize,
                                           int num_bits,bool linLayer_flag){{
    long int inputArrayLen = inputD*inputH*inputW;
    
    long int numNonZero = 0;
    __m256 acc0 = {{0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}};
    for (long int i = 0; i < 8*(inputArrayLen/8); i+=8){{
        __m256 cur = {{PADDINGMASK[i],PADDINGMASK[i+1],PADDINGMASK[i+2],PADDINGMASK[i+3],PADDINGMASK[i+4],PADDINGMASK[i+5],PADDINGMASK[i+6],PADDINGMASK[i+7]}};
        acc0 = _mm256_add_ps(acc0,cur);
    }}
    float* ptrToAcc0 = (float*)&acc0;
    for (int i = 0; i < 8; i++){{
        numNonZero += *ptrToAcc0;
        ptrToAcc0 += 1;
    }}
    for (long int i = 8*(inputArrayLen/8); i < inputArrayLen; i++){{
        numNonZero += PADDINGMASK[i];
    }}
    float mean_ = 0;
    __m256 acc = {{0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}};
    for (long int i = 0; i < 8*(inputArrayLen/8); i+=8){{
        __m256 cur = {{INPUTARRAY[i],INPUTARRAY[i+1],INPUTARRAY[i+2],INPUTARRAY[i+3],INPUTARRAY[i+4],INPUTARRAY[i+5],INPUTARRAY[i+6],INPUTARRAY[i+7]}};
        acc = _mm256_add_ps(acc,cur);
    }}
    float* ptrToAcc = (float*)&acc;
    for (int i = 0; i < 8; i++){{
        mean_ += *ptrToAcc;
        ptrToAcc += 1;
    }}
    for (long int i = 8*(inputArrayLen/8); i < inputArrayLen; i++){{
        mean_ += INPUTARRAY[i];
    }}
    mean_ = mean_/(numNonZero+1e-5);
    float var_ = 0;

    __m256 acc1 = {{0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}};
    __m256 minus_means = {{-mean_, -mean_, -mean_, -mean_, -mean_, -mean_, -mean_, -mean_}};
    for (long int i = 0; i < 8*(inputArrayLen/8);i+=8){{
        __m256 cur = {{INPUTARRAY[i],INPUTARRAY[i+1],INPUTARRAY[i+2],INPUTARRAY[i+3],INPUTARRAY[i+4],INPUTARRAY[i+5],INPUTARRAY[i+6],INPUTARRAY[i+7]}};
        cur = _mm256_add_ps(cur, minus_means);
        cur = _mm256_mul_ps(cur,cur);
        __m256 p_mask = {{PADDINGMASK[i],PADDINGMASK[i+1],PADDINGMASK[i+2],PADDINGMASK[i+3],PADDINGMASK[i+4],PADDINGMASK[i+5],PADDINGMASK[i+6],PADDINGMASK[i+7]}};
        cur = _mm256_mul_ps(cur, p_mask);
        acc1 = _mm256_add_ps(acc1, cur);
    }}
    float* ptrToAcc1 = (float*)&acc1;
    for (int i = 0; i < 8; i++){{
        var_ += *ptrToAcc1;
        ptrToAcc1 += 1;
    }}
    for (long int i = 8*(inputArrayLen/8); i < inputArrayLen; i++){{
        var_ += PADDINGMASK[i] * pow((INPUTARRAY[i]-mean_),2);
    }}

          
    var_ = var_/(numNonZero-1);
    float denom = sqrt(var_+1e-5);

    __m256 inv_denoms = {{1/denom, 1/denom, 1/denom, 1/denom, 1/denom, 1/denom, 1/denom, 1/denom}};
    for (long int i = 0; i < 8*(inputArrayLen/8); i+=8){{
        __m256 cur = {{INPUTARRAY[i],INPUTARRAY[i+1],INPUTARRAY[i+2],INPUTARRAY[i+3],INPUTARRAY[i+4],INPUTARRAY[i+5],INPUTARRAY[i+6],INPUTARRAY[i+7]}};
        cur = _mm256_add_ps(cur, minus_means);
        cur = _mm256_mul_ps(cur, inv_denoms);
        __m256 p_mask = {{PADDINGMASK[i],PADDINGMASK[i+1],PADDINGMASK[i+2],PADDINGMASK[i+3],PADDINGMASK[i+4],PADDINGMASK[i+5],PADDINGMASK[i+6],PADDINGMASK[i+7]}};
        cur = _mm256_mul_ps(cur, p_mask);
        _mm256_store_ps(&INPUTARRAY[i], cur);
        //INPUTARRAY[i] = PADDINGMASK[i] * (INPUTARRAY[i]-mean_)/denom;
    }}
    
    for (long int i = 8*(inputArrayLen/8); i < inputArrayLen; i++){{
        INPUTARRAY[i] = PADDINGMASK[i] * (INPUTARRAY[i]-mean_)/denom;
    }}
    
    float absmax = fabsf(INPUTARRAY[0]);
    for (long int i = 1; i < inputArrayLen; i++){{fabsf(INPUTARRAY[i]) > absmax ? absmax = fabsf(INPUTARRAY[i]) : 0;}}






    float quantizationRange = pow(2,(num_bits-1));
    float minclamp  = -quantizationRange+1e-5;
    float maxclamp = +quantizationRange+1e-5;
    float factor_quant = quantizationRange/absmax;
    __m256 factor_quant_vec = {{factor_quant,factor_quant,factor_quant,factor_quant,factor_quant,factor_quant,factor_quant,factor_quant}};
    __m256 minclamp_vec = {{minclamp,minclamp,minclamp,minclamp,minclamp,minclamp,minclamp,minclamp}};
    __m256 maxclamp_vec = {{maxclamp,maxclamp,maxclamp,maxclamp,maxclamp,maxclamp,maxclamp,maxclamp}};
    for (long int i = 0; i < 8*(inputArrayLen/8); i+=8){{
        __m256 cur = {{INPUTARRAY[i],INPUTARRAY[i+1],INPUTARRAY[i+2],INPUTARRAY[i+3],INPUTARRAY[i+4],INPUTARRAY[i+5],INPUTARRAY[i+6],INPUTARRAY[i+7]}};
        cur = _mm256_mul_ps(cur, factor_quant_vec);
        cur = _mm256_max_ps(cur, minclamp_vec);
        cur = _mm256_min_ps(cur, maxclamp_vec);
        _mm256_store_ps(&INPUTARRAY[i], cur);
        //cur = min(max(cur,minclamp) , maxclamp);
    }}
    
    for (long int i = 8*(inputArrayLen/8); i < inputArrayLen; i++){{
        INPUTARRAY[i] = min(max(INPUTARRAY[i],minclamp), maxclamp);
    }}
    
    int outputD = out_channels;
    int outputH = (inputH-kernelSize)/stride+1+2*output_padding; 
    int outputW = (inputW-kernelSize)/stride+1+2*output_padding;
    if (linLayer_flag){{
        outputH = 1;
        outputW = 1;
    }}
    if (bias_flag){{
        for (int d = 0; d < outputD; d++){{
            for (int i = output_padding; i < outputH-output_padding; i++){{
                for (int j = output_padding; j < outputW-output_padding; j++){{
                    OUTPUTARRAY[iib(d,i,j,outputH,outputW)] = BIAS[d];
                    PADDINGMASK[iib(d,i,j,outputH,outputW)] = 1;}}
            }}
        }}
    }}
    if (!bias_flag){{
        for (int d = 0; d < outputD; d++){{
            for (int i = output_padding; i < outputH-output_padding; i++){{
                for (int j = output_padding; j < outputW-output_padding; j++){{
                    OUTPUTARRAY[iib(d,i,j,outputH,outputW)] = 0;
                    PADDINGMASK[iib(d,i,j,outputH,outputW)] = 1;}}
            }}
        }}
    }}
    if (output_padding > 0){{
        padNext(OUTPUTARRAY,PADDINGMASK,output_padding,outputD,outputH,outputW);}}
    return absmax;
}}

void dequantRelu(float ARRAY[],bool PADDINGMASK[],
                 long int arrayLen,float absmax,
                 float absmeanweight,int num_bits,
                 float neg_slope,bool non_linearity){{
    float quantizationRange = pow(2,(num_bits - 1));
    float factor = absmax*absmeanweight/quantizationRange;
    __m256 factor_vec = {{factor, factor, factor, factor, factor, factor, factor, factor}};
    for (long int i = 0; i < 8*(arrayLen/8); i+=8){{
        __m256 cur = {{ARRAY[i],ARRAY[i+1],ARRAY[i+2],ARRAY[i+3],ARRAY[i+4],ARRAY[i+5],ARRAY[i+6],ARRAY[i+7]}};
        cur = _mm256_mul_ps(cur, factor_vec);
        _mm256_store_ps(&ARRAY[i], cur);
    	//PADDINGMASK[i] ? ARRAY[i] = ARRAY[i]*factor : 0;
    }}
    
    for (long int i = 8*(arrayLen/8); i < arrayLen; i++){{
        ARRAY[i] = ARRAY[i]*factor;
        //PADDINGMASK[i] ? ARRAY[i] = ARRAY[i]*factor : 0;
    }}
    
    if (non_linearity){{
        __m256 neg_slope_vec = {{neg_slope, neg_slope, neg_slope, neg_slope, neg_slope, neg_slope, neg_slope, neg_slope}};
        __m256 zero_vec = {{0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}};
        for (long int i = 0; i < 8*(arrayLen/8); i+=8){{
            __m256 cur = {{ARRAY[i],ARRAY[i+1],ARRAY[i+2],ARRAY[i+3],ARRAY[i+4],ARRAY[i+5],ARRAY[i+6],ARRAY[i+7]}};
            __m256 minarr = _mm256_min_ps(zero_vec, cur);
            minarr = _mm256_mul_ps(minarr, neg_slope_vec);
            __m256 maxarr = _mm256_max_ps(zero_vec, cur);
            __m256 rest = _mm256_add_ps(minarr, maxarr);
            _mm256_store_ps(&ARRAY[i], rest);
    	    //PADDINGMASK[i] ? ARRAY[i] = max(0,ARRAY[i])+neg_slope*min(0,ARRAY[i]) : 0;
        }}
        
        for (long int i = 8*(arrayLen/8); i < arrayLen; i++){{
            ARRAY[i] = max(0,ARRAY[i])+neg_slope*min(0,ARRAY[i]);
        }}
        
    }}               
}}


void maxpool2d(int input_padding,float INPUTARRAY[],
               int D,int inputH,int inputW,
               int kernel_size,int stride,
               int output_padding,float OUTPUTARRAY[],
               bool PADDINGMASK[]){{
    int outputD = D;
    int outputHnp = (inputH-2*input_padding-kernel_size)/stride+1;//I HOPE THIS DIVISION WON'T CAUSE PROBLEMS
    int outputWnp = (inputW-2*input_padding-kernel_size)/stride+1;
    int outputH = outputHnp+2*output_padding;
    int outputW = outputWnp+2*output_padding;
    for (long int i = 0; i < outputD*outputH*outputW; i++){{
        PADDINGMASK[i] = 1;}}
    if (output_padding > 0){{
        padNext(OUTPUTARRAY,PADDINGMASK,output_padding,outputD,outputH,outputW);}}
    int iin = input_padding;
    int jin = input_padding;
    for (int d = 0; d < outputD; d++){{
        for (int iout = output_padding; iout < outputH-output_padding; iout++){{
            for (int jout = output_padding; jout < outputW-output_padding; jout++){{
                float maxim = INPUTARRAY[iib(d,iin,jin,inputH,inputW)];
                for (int rowiter = iin; rowiter < iin+kernel_size; rowiter++){{
                    for (int coliter = jin; coliter < jin+kernel_size; coliter++){{
                        if (INPUTARRAY[iib(d,rowiter,coliter,inputH,inputW)] > maxim){{
                            maxim = INPUTARRAY[iib(d,rowiter,coliter,inputH,inputW)];}}
                    }}
                }}
                OUTPUTARRAY[iib(d,iout,jout,outputH,outputW)] = maxim;
                jin += stride;
            }}
            iin += stride;
            jin = input_padding;
        }}
        iin = input_padding;
    }}
}}

void copyWithNoPad(float INPUTARRAY[],float OUTPUTARRAY[],bool PADDINGMASK[],int D,int H,int W,int pad){{
    long int origArrayLen = D*H*W;
    long int newArrayLen = D*(H-2*pad)*(W-2*pad);
    long int j = 0;
    for (long int i = 0; i < origArrayLen; i++){{
        if (PADDINGMASK[i] == 1){{
            OUTPUTARRAY[j] = INPUTARRAY[i];
            j+=1;}}
    }}
    for (long int j = 0; j < newArrayLen; j++){{
        PADDINGMASK[j] = 1;}}
}}

/*
INSERT YOUR FUNCTION DEFINITIONS HERE
*/


int main() {{
/*
INSERT YOU BIAS INIT HERE
*/

  FILE *fileArr,*fileMask;
  fileArr = fopen(INPUTFILENAME,"r");
  fileMask = fopen(PADMASKFILENAME,"r");
  for (int k = 0; k < STARRLEN; k++){{
    fscanf(fileArr,"%f", &A[k]);
    fscanf(fileMask,"%d", &P[k]);
  }}
  fclose(fileArr);
  fclose(fileMask);
  clock_t begin = clock();

/*
INSERT YOUR FUNCTION CALLS HERE
*/
  
  clock_t end = clock();
  double total_time = (double)(end - begin) / CLOCKS_PER_SEC;
  time_spent_in_conv = time_spent_in_conv/CLOCKS_PER_SEC;
  FILE *fileLog = fopen(LOGFILENAME, "w");
  fprintf(fileLog,"CPU time spent: %f\\n",total_time);
  fprintf(fileLog,"Time spent in conv: %f\\n",time_spent_in_conv);
  fprintf(fileLog,"Time spent in conv: %f %% \\n",time_spent_in_conv/total_time*100);

  FILE *fileRes = fopen(OUTPUTFILENAME,"w");
  for (int k = 0; k < OUTPUTARRAYLEN; k++){{
    fprintf(fileRes,"%f\\n",{ARRFROMWHICHTOREADOUTPUT}[k]);
  }}
  fclose(fileRes);
  fclose(fileLog);
  //scanf("%d");
  return 0;
}}
'''

In [6]:
#print(zah)

In [7]:

def genBiasStructDeclaration(BIASES):
    ret = "struct bias_struct {\n"
    for i in range(0,len(BIASES)):
        if BIASES[i] == 0:
            ret += f"float b{i}[1];"
        else:
            thisLen = len(BIASES[i])
            ret += f"float b{i}[{thisLen}];"
    ret += "};\n"
    ret += "struct bias_struct BIASES;\n"
    return ret

def genPopulateBiasStruct(BIASES):
    ret = ""
    for i in range(0, len(BIASES)):
        if BIASES[i] == 0:
            ret += f"BIASES.b{i}[0] = 1.0;"
        else:
            for j in range(0, len(BIASES[i])):
                ret += f"BIASES.b{i}[{j}] = {BIASES[i][j]};"
    return ret + '\n'





def genOneConvFilterNOIIB(weight,dinin,dinout,inputH,inputW,INPUTARRAYNAME,OUTPUTARRAYNAME,outputH,outputW):
    kernel_depth = weight.shape[0]
    kernel_size = weight.shape[1]
    output = f'{OUTPUTARRAYNAME}[iib({dinout},iout,jout,{outputH},{outputW})]+='
    nonzerow = 0
    for dr in range(0,kernel_depth):
        for ir in range(0, kernel_size):
            for jr in range(0, kernel_size):
                if weight[dr,ir,jr] == 1:
                    output+=f'+{INPUTARRAYNAME}[fs+{dinin*inputH*inputW+dr*inputH*inputW+ir*inputW+jr}]'
                    nonzerow += 1
                if weight[dr,ir,jr] == -1:
                    output+=f'-{INPUTARRAYNAME}[fs+{dinin*inputH*inputW+dr*inputH*inputW+ir*inputW+jr}]'
                    nonzerow += 1
    output+='+0;\n'
    path_to_add_count = "conv_count.txt"
    if not os.path.exists(path_to_add_count):
        with open(path_to_add_count, 'w') as f:
            f.write('')
    else:
        with open(path_to_add_count, 'a') as f:
            f.write(str(nonzerow)+'\n')
    #print("NON ZERO WEIGHTS", nonzerow)
    return output

def genBitConvBitPythonNOIIB(weight,bias_flag,BIASARRAYNAME,
                             in_channels,out_channels,
                             kernel_size,stride,
                             dilation,groups,
                             INPUTARRAYNAME,
                             OUTPUTARRAYNAME, 
                             inputH,inputW,
                             output_padding,
                             PADDINGMASKNAME,
                             absmeanweight,
                             neg_slope,num_bits,
                             non_linearity):

    outputH = (inputH-kernel_size)//stride+1+2*output_padding 
    outputW = (inputW-kernel_size)//stride+1+2*output_padding
    #how will we call our funcntions????
    with open("curFNameNo.txt","r") as f:
        num = int(f.read())
    with open("curFNameNo.txt","w") as f:
        f.write(str(num+1))
    with open("layerNames.txt", "r") as f:
        allNames = f.read().split('\n')
        funcName = allNames[num]
    #what will be the function's interface?
    #layerName(I,O,ioutStart,ioutEndPlusOne,istart,jstart,stride)   
    fDefCode =f'''
void {funcName}(float I[],float O[],int ioutStart,int ioutEndPlusOne,int istart,int jstart){{
int i = istart;
int j = jstart;
'''
    fDefCode += f'for (int iout = ioutStart; iout < ioutEndPlusOne; iout++){{\n'
    fDefCode+=f'\tfor (int jout = {output_padding}; jout < {outputW-output_padding}; jout++){{\n\t\t'
    fDefCode+=f'long int fs = j+{inputW}*i;\n'
    filterDepth = in_channels//groups
    filtersPerGroup = out_channels//groups
    dinin = 0
    for dinout in range(0, out_channels):
        if dinout > 0 and dinout%filtersPerGroup==0:
            dinin += filterDepth
        fDefCode+= genOneConvFilterNOIIB(weight[dinout],
                                           dinin,dinout,
                                           inputH,inputW,
                                           'I',
                                           'O',
                                           outputH,outputW)
    fDefCode += f"\n\t\tj+={stride};}}\n\ti+={stride};\n\tj=0;}}\n}}\n"

    fCallCode = f'''
absmax = layernormQuantizationBiasNextpadding({in_channels},{inputH},{inputW},{out_channels},{stride},{INPUTARRAYNAME},{PADDINGMASKNAME},{BIASARRAYNAME},{OUTPUTARRAYNAME},{output_padding},{bias_flag},{kernel_size},{num_bits},false);
st = clock();
{funcName}({INPUTARRAYNAME},{OUTPUTARRAYNAME},{output_padding},{outputH-output_padding},0,0);
end = clock();
time_spent_in_conv += (double)(end - st);
dequantRelu({OUTPUTARRAYNAME},{PADDINGMASKNAME},{out_channels*outputH*outputW},absmax,{absmeanweight},{num_bits},{neg_slope},{non_linearity});
'''
    return {'fDef': fDefCode, 'fCall': fCallCode}


def genPoolCodePython(input_padding,maxOrAvg,prevfDefs,prevfCalls,
                      INPUTARRAY,D,
                      inputH,inputW,
                      kernel_size,stride,
                      output_padding,
                      OUTPUTARRAY,PADDINGMASK):
    outputH=(inputH-2*input_padding-kernel_size)//stride+1+2*output_padding
    outputW=(inputW-2*input_padding-kernel_size)//stride+1+2*output_padding
    ret = f"{maxOrAvg}({input_padding},{INPUTARRAY},{D},{inputH},{inputW},{kernel_size},{stride},{output_padding},{OUTPUTARRAY},{PADDINGMASK});\n"
    return {'inputH': outputH,'inputW': outputW,'fDefs': prevfDefs, 'fCalls': prevfCalls + ret + '\n'}



def genBitLinearBitPython(weight,bias_flag,BIASARRAYNAME,in_channels,out_channels,
                        INPUTARRAYNAME,OUTPUTARRAYNAME, 
                        inputH,inputW,input_padding,PADDINGMASKNAME,
                        absmeanweight,neg_slope,num_bits,non_linearity):

    #how will we call our funcntions????
    with open("curFNameNo.txt","r") as f:
        num = int(f.read())
    with open("curFNameNo.txt","w") as f:
        f.write(str(num+1))
    with open("layerNames.txt", "r") as f:
        allNames = f.read().split('\n')
        funcName = allNames[num]
    #what will be the function's interface?
    #layerName(I,O,ioutStart,ioutEndPlusOne,istart,jstart,stride)
    fDefCode = f'''
void {funcName}(float I[],float O[]){{
'''
    inputArrayLen = in_channels*(inputH-2*input_padding)*(inputW-2*input_padding)
    for j in range(0,out_channels):
        fDefCode+=f'O[{j}]+='#copy with no pad causes this confusing reversal
        for it in range(0,inputArrayLen):
            if weight[j][it] == 1:
                fDefCode+=f'+I[{it}]'
            if weight[j][it] == -1:
                fDefCode+=f'-I[{it}]'
        fDefCode+='+0;'
    fDefCode+=f'}}'

    fCallCode = f'''
copyWithNoPad({INPUTARRAYNAME},{OUTPUTARRAYNAME},{PADDINGMASKNAME},{in_channels},{inputH},{inputW},{input_padding});
absmax = layernormQuantizationBiasNextpadding({in_channels},{inputH-2*input_padding},{inputW-2*input_padding},{out_channels},1,{OUTPUTARRAYNAME},{PADDINGMASKNAME},{BIASARRAYNAME},{INPUTARRAYNAME},0,{bias_flag},1,{num_bits},true);
st = clock();
{funcName}({OUTPUTARRAYNAME},{INPUTARRAYNAME});
end = clock();
time_spent_in_conv += (double)(end - st);
dequantRelu({INPUTARRAYNAME},{PADDINGMASKNAME},{out_channels*1*1},absmax,{absmeanweight},{num_bits},{neg_slope},{non_linearity});
'''
    return {'fDef': fDefCode, 'fCall': fCallCode}



def genCodeDS(prevfDefs,prevfCalls,W1,AMW1,B1NAME,W2,AMW2,B2NAME,BFL,
              in_channels,out_channels,
       kernel_size,stride,groups,INPUTARRAYNAME,INTERARRAYNAME,OUTPUTARRAYNAME,
       PADDINGMASKNAME,output_padding,inputH,inputW):
    depth_code = genBitConvBitPythonNOIIB(weight=W1,bias_flag=BFL,
                                          BIASARRAYNAME=B1NAME,
                             in_channels=in_channels,
                             out_channels=in_channels,
                             kernel_size=kernel_size,
                             stride=stride,
                             dilation=1,
                             groups=in_channels,
                             INPUTARRAYNAME=INPUTARRAYNAME,
                             OUTPUTARRAYNAME=INTERARRAYNAME, 
                             inputH=inputH,inputW=inputW,
                             output_padding=0,
                             PADDINGMASKNAME=PADDINGMASKNAME,
                             absmeanweight=AMW1,
                             neg_slope=0.1,num_bits=8,
                             non_linearity='false')#False
    inputH_new = (inputH-kernel_size)//stride+1 
    inputW_new = (inputW-kernel_size)//stride+1 
    sep_code = genBitConvBitPythonNOIIB(weight=W2,bias_flag=BFL,
                                        BIASARRAYNAME=B2NAME,
                             in_channels=in_channels,
                             out_channels=out_channels,
                             kernel_size=1,stride=1,
                             dilation=1,groups=1,
                             INPUTARRAYNAME=INTERARRAYNAME,
                             OUTPUTARRAYNAME=OUTPUTARRAYNAME, 
                             inputH=inputH_new,inputW=inputW_new,
                             output_padding=output_padding,
                             PADDINGMASKNAME=PADDINGMASKNAME,
                             absmeanweight=AMW2,
                             neg_slope=0.1,num_bits=8,
                             non_linearity='true')#True
    outputH = (inputH_new-1)//1+1 + 2*output_padding
    outputW = (inputW_new-1)//1+1 + 2*output_padding
    return {'inputH':outputH,
            'inputW':outputW,
            'fDefs':prevfDefs+depth_code['fDef']+sep_code['fDef'],
            'fCalls':prevfCalls+depth_code['fCall']+sep_code['fCall']}

class CNNBLOCK_DSBitGenCode(nn.Module):#i should use more *ags and **kwargs
    def __init__(self, weightAbsMeanD,weightAbsMeanS,in_channels_, out_channels_,
                 kernel_size_, stride_, 
                 padding_, bias_,nextPadding,IAN,INTERAN,OAN,PMN,B1NAME,B2NAME):
        super().__init__()
        self.AMW1,self.AMW2 = weightAbsMeanD,weightAbsMeanS
        self.BFL = bias_
        biasAsVal = False
        if bias_ == 'true':
            biasAsVal = True
        self.in_channels,self.out_channels = in_channels_,out_channels_
        self.kernel_size,self.stride = kernel_size_,stride_
        self.next_padding = nextPadding
        self.INPUTARRAYNAME = IAN
        self.INTERARRAYNAME = INTERAN
        self.OUTPUTARRAYNAME = OAN
        self.PADDINGMASKNAME = PMN
        self.B1NAME = B1NAME
        self.B2NAME = B2NAME
        self.conv_depth = BitConv2dBitProba(weightAbsMean = weightAbsMeanD,in_channels = in_channels_, 
                                    out_channels =in_channels_, 
                                    kernel_size =kernel_size_, 
                                    stride = stride_, 
                                    padding = padding_, 
                                    dilation = 1,
                                    groups =in_channels_,
                                    bias = biasAsVal)
        self.conv_sep = BitConv2dBitProba(weightAbsMean = weightAbsMeanS,in_channels = in_channels_, 
                                  out_channels = out_channels_, 
                                  kernel_size=1, 
                                  stride = 1,
                                  padding= 0, 
                                  dilation=1,
                                  groups = 1,
                                  bias=False)
        self.lrlu = nn.LeakyReLU(0.1)
    def forward(self, x):
        return genCodeDS(prevfDefs=x['fDefs'],
                         prevfCalls=x['fCalls'],
                  W1=self.conv_depth.weight,
                  AMW1=self.AMW1,
                  B1NAME=self.B1NAME,
                  W2=self.conv_sep.weight,
                  AMW2=self.AMW2,
                  B2NAME=self.B2NAME,
                  BFL=self.BFL,
                  in_channels=self.in_channels,
                  out_channels=self.out_channels,
                  kernel_size=self.kernel_size,
                  stride=self.stride,
                  groups=self.in_channels,
                  INPUTARRAYNAME=self.INPUTARRAYNAME,
                  INTERARRAYNAME=self.INTERARRAYNAME,
                  OUTPUTARRAYNAME=self.OUTPUTARRAYNAME,
                  PADDINGMASKNAME=self.PADDINGMASKNAME,
                  output_padding=self.next_padding,
                  inputH=x['inputH'],inputW=x['inputW'])#or maybe biases should be external...

class BitLinearBitGenCode(nn.Linear):
    def __init__(self,bias_flag,BIASARRAYNAME,in_channels,out_channels,
                        INPUTARRAYNAME,OUTPUTARRAYNAME, 
                        input_padding,PADDINGMASKNAME,
                        absmeanweight,neg_slope,num_bits,non_linearity):
        bias_as_val = False
        if bias_flag == 'true':
            bias_as_val = True
        super().__init__(in_channels,out_channels,bias_as_val)
        self.bias_flag = bias_flag
        self.BIASARRAYNAME = BIASARRAYNAME
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.INPUTARRAYNAME = INPUTARRAYNAME
        self.OUTPUTARRAYNAME = OUTPUTARRAYNAME
        self.input_padding = input_padding
        self.PADDINGMASKNAME = PADDINGMASKNAME
        self.absmeanweight = absmeanweight
        self.neg_slope = neg_slope
        self.num_bits = num_bits
        self.non_linearity = non_linearity
    def forward(self,x):
        resultDict = genBitLinearBitPython(weight=self.weight,
                bias_flag=self.bias_flag,
                BIASARRAYNAME=self.BIASARRAYNAME,
                in_channels=self.in_channels,
                out_channels=self.out_channels,
                INPUTARRAYNAME=self.INPUTARRAYNAME,
                OUTPUTARRAYNAME=self.OUTPUTARRAYNAME, 
                inputH=x['inputH'],
                inputW=x['inputW'],
                input_padding=self.input_padding,
                PADDINGMASKNAME=self.PADDINGMASKNAME,
                absmeanweight=self.absmeanweight,
                neg_slope=self.neg_slope,
                num_bits=self.num_bits,
                non_linearity=self.non_linearity)
        return {'inputH':1,'outputH':1,
                'fCalls':x['fCalls']+resultDict['fCall'],
                'fDefs':x['fDefs']+resultDict['fDef']}

class RESBLOCKBitGenCode(nn.Module):
    def __init__(self, ANTC,list_of_params):
        super().__init__()
        modules = []
        self.ARRAYNAMETOCOPY = ANTC
        self.list_of_params = list_of_params
        self.STARTINGARRAY = list_of_params[0][9]
        self.NAMEOFARRAYTOSAVETO = list_of_params[0][9]
        self.inputDim = list_of_params[0][2]
        self.NAMEOFTHELASTARRAY = list_of_params[-1][11]
        for block in self.list_of_params:
            modules.append(CNNBLOCK_DSBitGenCode(weightAbsMeanD=block[0],
                                          weightAbsMeanS=block[1],
                                          in_channels_=block[2],
                                          out_channels_=block[3],
                                          kernel_size_=block[4], 
                                          stride_=block[5], 
                                          padding_=block[6], 
                                          bias_=block[7],
                                          nextPadding=block[8],
                                          IAN=block[9],
                                          INTERAN=block[10],
                                          OAN=block[11],
                                          PMN=block[12],
                                          B1NAME=block[13],
                                          B2NAME=block[14]))
        self.suka = nn.Sequential(*modules)
    def forward(self, x):
        inputH,inputW = x['inputH'],x['inputW']
        prevfCalls = x['fCalls']
        prevfDefs = x['fDefs']
        #theFirstBit = prevfCalls+f"\nfor (i = 0; i < {self.inputDim*inputH*inputW}; i++){{\n\t{self.ARRAYNAMETOCOPY}[i] = {self.STARTINGARRAY}[i];}}\n"
        theFirstBit = prevfCalls+f"memcpy({self.ARRAYNAMETOCOPY}, {self.STARTINGARRAY}, {self.inputDim*inputH*inputW}*sizeof(float));\n"
        almostThere = self.suka({'inputH':inputH,'inputW':inputW,'fCalls':'','fDefs':''})
        #theLastBit = f"\nfor (i = 0; i < {self.inputDim*almostThere['inputH']*almostThere['inputW']}; i++){{\n\t{self.NAMEOFTHELASTARRAY}[i]+={self.ARRAYNAMETOCOPY}[i];}}\n"
        theLastBit = f"for (i = 0; i < 8*({self.inputDim*almostThere['inputH']*almostThere['inputW']}/8); i+=8){{\n"
        theLastBit += f"__m256 cur0 = _mm256_load_ps(&{self.NAMEOFTHELASTARRAY}[i]);\n"
        theLastBit += f"__m256 cur1 = _mm256_load_ps(&{self.ARRAYNAMETOCOPY}[i]);\n"
        theLastBit += f"cur0 = _mm256_add_ps(cur0, cur1);\n"
        theLastBit += f"_mm256_store_ps(&{self.NAMEOFTHELASTARRAY}[i], cur0);\n}}\n"
        theLastBit += f"for (i = 8*({self.inputDim*almostThere['inputH']*almostThere['inputW']}/8); i < {self.inputDim*almostThere['inputH']*almostThere['inputW']}; i++){{\n"
        theLastBit += f"\t{self.NAMEOFTHELASTARRAY}[i]+={self.ARRAYNAMETOCOPY}[i];}}\n"
        return {'inputH':almostThere['inputH'],
                'inputW':almostThere['inputW'],
                'fCalls':theFirstBit+almostThere['fCalls']+theLastBit,
                'fDefs':prevfDefs+almostThere['fDefs']}#and here we'll have the operation of addition...


class YOLONETBitGenCode(nn.Module):#now let's modify it.....
    def __init__(self, lowam,arrAName,arrBName,copyArrName,padArrName):
        super().__init__()
        self.pool = nn.MaxPool2d(2,2)       
        self.arrA = arrAName
        self.arrB = arrBName
        self.arrC = copyArrName
        self.arrP = padArrName
        self.convBlock1 = CNNBLOCK_DSBitGenCode(lowam[0],lowam[1],3, 64, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{0}',f'BIASES.b{1}')       
        self.resblock64_1 = RESBLOCKBitGenCode(self.arrC,[[lowam[2],lowam[3],64, 64, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{2}',f'BIASES.b{3}'], [lowam[4],lowam[5],64,64, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{4}',f'BIASES.b{5}'] ])
        self.resblock64_2 = RESBLOCKBitGenCode(self.arrC,[[lowam[6],lowam[7],64, 64, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{6}',f'BIASES.b{7}'], [lowam[8],lowam[9],64,64, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{8}',f'BIASES.b{9}'] ])
        self.resblock64_3 = RESBLOCKBitGenCode(self.arrC,[[lowam[10],lowam[11],64, 64, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{10}',f'BIASES.b{11}'], [lowam[12],lowam[13],64,64, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{12}',f'BIASES.b{13}'] ])#????????????????????????????????????????????????????????????????????????????? What's the problem here???
        #pool here A0 --> A1
        self.resblock64_4 = RESBLOCKBitGenCode(self.arrC,[[lowam[14],lowam[15],64, 64, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{14}',f'BIASES.b{15}'], [lowam[16],lowam[17],64,64, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{16}',f'BIASES.b{17}'] ])
        self.resblock64_5 = RESBLOCKBitGenCode(self.arrC,[[lowam[18],lowam[19],64, 64, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{18}',f'BIASES.b{19}'], [lowam[20],lowam[21],64,64, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{20}',f'BIASES.b{21}'] ])
        #pool here A1 --> A0
        self.convInter_64_128 = CNNBLOCK_DSBitGenCode(lowam[22],lowam[23],64, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{22}',f'BIASES.b{23}')
        self.resblock128_1 = RESBLOCKBitGenCode(self.arrC,[[lowam[24],lowam[25],128, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{24}',f'BIASES.b{25}'], [lowam[26],lowam[27],128, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{26}',f'BIASES.b{27}']])
        self.resblock128_2 = RESBLOCKBitGenCode(self.arrC,[[lowam[28],lowam[29],128, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{28}',f'BIASES.b{29}'], [lowam[30],lowam[31],128, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{30}',f'BIASES.b{31}']])
        #pool here A0 --> A1
        self.resblock128_3 = RESBLOCKBitGenCode(self.arrC,[[lowam[32],lowam[33],128, 128, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{32}',f'BIASES.b{33}'], [lowam[34],lowam[35],128, 128, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{34}',f'BIASES.b{35}']])
        #pool here A1 --> A0
        self.resblock128_4 = RESBLOCKBitGenCode(self.arrC,[[lowam[36],lowam[37],128, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{36}',f'BIASES.b{37}'], [lowam[38],lowam[39],128, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{38}',f'BIASES.b{39}']])
        self.resblock128_5 = RESBLOCKBitGenCode(self.arrC,[[lowam[40],lowam[41],128, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{40}',f'BIASES.b{41}'], [lowam[42],lowam[43],128, 128, 3,1,1,'false',1,self.arrA,self.arrB,self.arrA,self.arrP,f'BIASES.b{42}',f'BIASES.b{43}']])
        #pool here A0 --> A1
        self.convInter_128_256 = CNNBLOCK_DSBitGenCode(lowam[44],lowam[45],128, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{44}',f'BIASES.b{45}')
        self.resblock256_1 = RESBLOCKBitGenCode(self.arrC,[[lowam[46],lowam[47],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{46}',f'BIASES.b{47}'], [lowam[48],lowam[49],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{48}',f'BIASES.b{49}']])
        self.resblock256_2 = RESBLOCKBitGenCode(self.arrC,[[lowam[50],lowam[51],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{50}',f'BIASES.b{51}'], [lowam[52],lowam[53],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{52}',f'BIASES.b{53}']])
        self.resblock256_3 = RESBLOCKBitGenCode(self.arrC,[[lowam[54],lowam[55],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{54}',f'BIASES.b{55}'], [lowam[56],lowam[57],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{56}',f'BIASES.b{57}']])
        self.resblock256_4 = RESBLOCKBitGenCode(self.arrC,[[lowam[58],lowam[59],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{58}',f'BIASES.b{59}'], [lowam[60],lowam[61],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{60}',f'BIASES.b{61}']])
        self.resblock256_5 = RESBLOCKBitGenCode(self.arrC,[[lowam[62],lowam[63],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{62}',f'BIASES.b{63}'], [lowam[64],lowam[65],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{64}',f'BIASES.b{65}']])
        self.resblock256_6 = RESBLOCKBitGenCode(self.arrC,[[lowam[66],lowam[67],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{66}',f'BIASES.b{67}'], [lowam[68],lowam[69],256, 256, 3,1,1,'false',1,self.arrB,self.arrA,self.arrB,self.arrP,f'BIASES.b{68}',f'BIASES.b{69}']])
        self.avgPool = nn.AvgPool2d(2,3)
        self.fc = BitLinearBitGenCode('true',f'BIASES.b{70}',256,10,
                        self.arrB,self.arrA, 
                        1,self.arrP,
                        lowam[70],0.1,8,'false')#(lowam[70],256, 10)
        #print('aaaaa suka', self.fc.weight)

    def forward(self,x):
        print('started generating code')
        x = self.convBlock1(x)
        x = self.resblock64_1(x)
        x = self.resblock64_2(x)
        x = self.resblock64_3(x)
        x = genPoolCodePython(input_padding=1,maxOrAvg='maxpool2d',prevfDefs = x['fDefs'],prevfCalls = x['fCalls'],INPUTARRAY=self.arrA,D=64,inputH=x['inputH'],inputW=x['inputW'],kernel_size=2,stride=2,output_padding=1,OUTPUTARRAY=self.arrB,PADDINGMASK=self.arrP)
        x = self.resblock64_4(x)
        x = self.resblock64_5(x)
        x = genPoolCodePython(input_padding=1,maxOrAvg='maxpool2d',prevfDefs = x['fDefs'],prevfCalls = x['fCalls'],INPUTARRAY=self.arrB,D=64,inputH=x['inputH'],inputW=x['inputW'],kernel_size=2,stride=2,output_padding=1,OUTPUTARRAY=self.arrA,PADDINGMASK=self.arrP)
        x = self.convInter_64_128(x)
        x = self.resblock128_1(x)
        x = self.resblock128_2(x)
        x = genPoolCodePython(input_padding=1,maxOrAvg='maxpool2d',prevfDefs = x['fDefs'],prevfCalls = x['fCalls'],INPUTARRAY=self.arrA,D=128,inputH=x['inputH'],inputW=x['inputW'],kernel_size=2,stride=2,output_padding=1,OUTPUTARRAY=self.arrB,PADDINGMASK=self.arrP)
        x = self.resblock128_3(x)
        x = genPoolCodePython(input_padding=1,maxOrAvg='maxpool2d',prevfDefs = x['fDefs'],prevfCalls = x['fCalls'],INPUTARRAY=self.arrB,D=128,inputH=x['inputH'],inputW=x['inputW'],kernel_size=2,stride=2,output_padding=1,OUTPUTARRAY=self.arrA,PADDINGMASK=self.arrP)
        x = self.resblock128_4(x)
        x = self.resblock128_5(x)
        x = genPoolCodePython(input_padding=1,maxOrAvg='maxpool2d',prevfDefs = x['fDefs'],prevfCalls = x['fCalls'],INPUTARRAY=self.arrA,D=128,inputH=x['inputH'],inputW=x['inputW'],kernel_size=2,stride=2,output_padding=1,OUTPUTARRAY=self.arrB,PADDINGMASK=self.arrP)
        x = self.convInter_128_256(x)
        x = self.resblock256_1(x)
        x = self.resblock256_2(x)
        x = self.resblock256_3(x)
        x = self.resblock256_4(x)
        x = self.resblock256_5(x)
        x = self.resblock256_6(x)
        x = self.fc(x)
        return x

In [8]:
def tensorCMP(t1,t2,eps):
    return torch.all(torch.abs(t1-t2) < eps).item()
def tensorMDP(tcorrect,t2):
    absDif = torch.abs(tcorrect-t2)
    meandif = torch.mean(absDif).item()
    meanabsval = torch.mean(torch.abs(tcorrect)).item()
    percent = 100*meandif/(meanabsval+1e-7)
    themax = torch.max(absDif)
    return {'mean%':percent,
            'max%':100*themax.item()/meanabsval}

In [9]:
primeNet = YOLONET()
primeNet.load_state_dict(torch.load('theCIFARbitnetWithNoBatchNorm (1).pt',
                                   map_location=DEVICE))


nzero = 0
ntotal = 0
for layer, item in primeNet.state_dict().items():
    if layer.split('.')[-1] != 'bias':
        mean_abs_val = item.abs().mean().item()
        eps = 1e-7
        scaled_weights= item / (mean_abs_val + eps)
        bin_weights_no_grad = torch.clamp(torch.round(scaled_weights), min=-1, max=1)
        bin_weights_with_grad = (bin_weights_no_grad - item).detach() + item
        for iter in bin_weights_with_grad.flatten():
            if iter.item() == 0:
                nzero += 1
            ntotal += 1
print(ntotal, nzero)

1081627 273862


In [10]:
#let's recall how do we actually quantize weights....
if True:
    
    primeNet = YOLONET()
    primeNet.load_state_dict(torch.load('theCIFARbitnetWithNoBatchNorm (1).pt',
                                   map_location=DEVICE))
    BIASES = []
    lowam = []
    qWeights = OrderedDict()
    for layer,item in primeNet.state_dict().items():
        if layer.split('.')[-1] == 'bias':
            BIASES.pop(-1)
            BIASES.append(item)
            qWeights[layer] = item
        else:
            BIASES.append(0)#what to do with this silly problem introduced completely by myself??
            mean_abs_val = item.abs().mean().item()
            eps = 1e-7
            scaled_weights= item / (mean_abs_val + eps)
            bin_weights_no_grad = torch.clamp(torch.round(scaled_weights), min=-1, max=1)
            bin_weights_with_grad = (bin_weights_no_grad - item).detach() + item
            qWeights[layer] = bin_weights_with_grad
            lowam.append(mean_abs_val)
    #probaNet = YOLONETBitProba(lowam)
    #probaNet.load_state_dict(qWeights)
    genTextNet = YOLONETBitGenCode(lowam,'A','B','C','P')
    genTextNet.load_state_dict(qWeights)

    for i in range(0, len(BIASES)):
        if torch.is_tensor(BIASES[i]):
            temp = []
            for j in range(0,len(BIASES[i])):
                temp.append(BIASES[i][j].item())
            BIASES[i] = temp
    with open("layerNames.txt","w") as f:
        f.write('')
    for layer in genTextNet.state_dict().keys():
        if layer.split('.')[-1] == 'weight':
            with open("layerNames.txt","a") as f:
                f.write(layer.replace('.','_')+'\n')
    with open("curFNameNo.txt",'w') as f:
        f.write(str(0))

    D,Hnp,Wnp = 3,32,32
    pad = 1
    H,W = Hnp+2*pad,Wnp+2*pad
    inputTensor = torch.rand((D,Hnp,Wnp))
    outputReal = torch.squeeze(primeNet(torch.unsqueeze(inputTensor,0)),0)
    if ALLLAYERS == False:
        outputRealPadded = torch.zeros((outputReal.shape[0], outputReal.shape[1]+2*pad, outputReal.shape[2]+2*pad))
        outputRealPadded[:,pad:pad+outputReal.shape[1],pad:pad+outputReal.shape[2]] = outputReal
        outputReal = outputRealPadded.clone()
    outputRealFlat = outputReal.flatten()
    with open("outputReal.txt", "w") as f:
        for i in range(0, len(outputRealFlat)):
            f.write(str(outputRealFlat[i].item())+'\n')
    #outputProba = probaNet(inputTensor)
    maxLen = 64*(32+2*pad)*(32+2*pad)#100_000#65536
    inputTensorPadded = torch.zeros((D,H,W))
    inputTensorPadded[:,pad:pad+Hnp,pad:pad+Wnp] = inputTensor.clone()
    At = torch.zeros(maxLen,dtype=torch.float32)
    At[0:D*H*W] = inputTensorPadded.flatten().clone()
    Bt = torch.zeros(maxLen,dtype=torch.float32)
    Ct = torch.zeros(maxLen,dtype=torch.float32)
    Pt = torch.ones(maxLen,dtype=torch.float32)
    padM = torch.zeros((D,H,W))
    padM[:,pad:pad+Hnp,pad:pad+Wnp] = torch.ones((D,Hnp,Wnp))
    Pt[0:D*H*W] = padM.flatten()
    A,B,C,P =[0]*maxLen,[0]*maxLen,[0]*maxLen,[1]*maxLen
    for i in range(0,maxLen):
        A[i],B[i],C[i],P[i] = At[i].item(),Bt[i].item(),Ct[i].item(),Pt[i].item()
    with open('inputTensorPaddedFlat.txt','w') as f:
        for i in range(0, D*H*W):
            f.write(str(A[i])+'\n')
    with open('paddingmask.txt','w') as f:
        for i in range(0, D*H*W):
            f.write(str(int(P[i]))+'\n')
    zah = zah0
    parts = zah.split("/*\nINSERT YOUR BIAS DECLARATION CODE HERE\n*/")
    bias_struct = genBiasStructDeclaration(BIASES)
    zah = parts[0] + bias_struct + parts[1]
    parts = zah.split("/*\nINSERT YOU BIAS INIT HERE\n*/")
    bias_filled = genPopulateBiasStruct(BIASES)
    zah = parts[0] + bias_filled + parts[1]

    outputMine = genTextNet({'inputH':34,'inputW':34,'fDefs':'','fCalls':''})
    print('gen is done')

    if os.path.exists('result_code_old.c'):
        os.remove('result_code_old.c')
    if os.path.exists('result_code_old.dll'):
        os.remove('result_code_old.dll')
    if os.path.exists('log.txt'):
        os.remove('log.txt')
    if os.path.exists('output.txt'):
        os.remove('output.txt')
    
    parts = zah.split("/*\nINSERT YOUR FUNCTION DEFINITIONS HERE\n*/")
    zah = parts[0] + outputMine['fDefs'] + parts[1]
    parts = zah.split("/*\nINSERT YOUR FUNCTION CALLS HERE\n*/")
    zah = parts[0] + outputMine['fCalls'] + parts[1]
    with open('result_code_old.c', 'w') as f:
        f.write(zah)

    #!gcc -march=x86-64-v3 -fPIC -shared -O2 -o C:/Users/oleks/Desktop1/AI_CODE_C/result_code.dll C:/Users/oleks/Desktop1/AI_CODE_C/result_code.c
    !cd /home/oleksiy-tsuber/AI_CODE_C
    !gcc -march=native -O3 -o result_code_old result_code_old.c -lm
    #!valgrind --tool=callgrind --dump-instr=yes --collect-jumps=yes --simulate-cache=yes ./result_code_old
    #-g flag makes so that debug info is present in the executable
    print('compilation done')
    if True:
        !./result_code_old
        print('all done')
        #clib = ctypes.CDLL("C:/Users/oleks/Desktop1/AI_CODE_C/result_code.dll")
        #clib.main()
        output = torch.as_tensor(np.loadtxt("output.txt"), dtype=torch.float32)
        print("OUTPUTLEN = ", len(output))
        print("REAL OUTPUT LEN = ", len(outputReal.flatten()))
        print(tensorMDP(outputReal.flatten(),output))
        with open("log.txt") as f:
            print(f.read())

started generating code
gen is done
result_code_old.c: In function ‘main’:
result_code_old.c:11768:23: warning: format ‘%d’ expects argument of type ‘int *’, but argument 3 has type ‘_Bool *’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
11768 |     fscanf(fileMask,"%d", &P[k]);
      |                      ~^   ~~~~~
      |                       |   |
      |                       |   _Bool *
      |                       int *
result_code_old.c:11767:5: warning: ignoring return value of ‘fscanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
11767 |     fscanf(fileArr,"%f", &A[k]);
      |     ^~~~~~~~~~~~~~~~~~~~~~~~~~~
result_code_old.c:11768:5: warning: ignoring return value of ‘fscanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8

In [11]:
inp = torch.rand((1,3,32,32))
st = time.time()
temp = primeNet(inp)
end = time.time()
print('time spent on real net: ', end - st)

time spent on real net:  0.0302274227142334


In [12]:
if True:
    n = 100
    sum0,sum1,sum2 = 0,0,0
    for i in range(0, n):
        if (i+1) % 1 == 0:
            print('cur iter = ', i+1)
        #clib.main()
        !cd /home/oleksiy-tsuber/AI_CODE_C
        !./result_code_old
        with open("log.txt") as f:
            text = f.read()
            parts = text.split('Time spent in conv: ')
            theBitWeCareAbout0 = parts[-1]
            theBitWeCareAbout0 = float(theBitWeCareAbout0.split(' %')[0])
            sum0 += theBitWeCareAbout0
            theBitWeCareAbout1 = parts[0]
            theBitWeCareAbout1 = float(theBitWeCareAbout1.split('CPU time spent: ')[1])
            sum1 += theBitWeCareAbout1
            
    s0,s1,s2 = sum0/n,sum1/n,sum2/n
    print('On avg we spent', s0, '% time in conv')
    print("Avg time for C bitnet:", s1)

cur iter =  1
cur iter =  2
cur iter =  3
cur iter =  4
cur iter =  5
cur iter =  6
cur iter =  7
cur iter =  8
cur iter =  9
cur iter =  10
cur iter =  11
cur iter =  12
cur iter =  13
cur iter =  14
cur iter =  15
cur iter =  16
cur iter =  17
cur iter =  18
cur iter =  19
cur iter =  20
cur iter =  21
cur iter =  22
cur iter =  23
cur iter =  24
cur iter =  25
cur iter =  26
cur iter =  27
cur iter =  28
cur iter =  29
cur iter =  30
cur iter =  31
cur iter =  32
cur iter =  33
cur iter =  34
cur iter =  35
cur iter =  36
cur iter =  37
cur iter =  38
cur iter =  39
cur iter =  40
cur iter =  41
cur iter =  42
cur iter =  43
cur iter =  44
cur iter =  45
cur iter =  46
cur iter =  47
cur iter =  48
cur iter =  49
cur iter =  50
cur iter =  51
cur iter =  52
cur iter =  53
cur iter =  54
cur iter =  55
cur iter =  56
cur iter =  57
cur iter =  58
cur iter =  59
cur iter =  60
cur iter =  61
cur iter =  62
cur iter =  63
cur iter =  64
cur iter =  65
cur iter =  66
cur iter =  67
cur 

In [13]:
if False:
    !valgrind --tool=callgrind --dump-instr=yes --collect-jumps=yes --simulate-cache=yes ./result_code_old